In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from datetime import datetime
import re 
from sqlalchemy import create_engine
import pyodbc
import pandas as pd

In [2]:
def scroll_to_quarter_page(driver):
    total_height = driver.execute_script("return document.body.scrollHeight")
    quarter_height = total_height // 4
    driver.execute_script(f"window.scrollTo(0, {quarter_height});")
    time.sleep(2)  

In [3]:
def get_sunrise_details(driver):
    sunrise=driver.find_elements(By.CLASS_NAME, "sunrise-sunset__times-value")[0].text
    sunset=driver.find_elements(By.CLASS_NAME, "sunrise-sunset__times-value")[1].text
    return sunrise,sunset

In [4]:
def get_forcasting_details(driver):
    date_day=[]
    temperature_max=[]
    temperature_min=[]
    description_day=[]
    rain=[]
    forcasting_list=driver.find_element(By.CLASS_NAME, "daily-list-body")
    forcasting_list_day=forcasting_list.find_elements(By.CLASS_NAME, "daily-list-item ")
    for i in forcasting_list_day[1:]:
        date_day.append(i.find_element(By.CLASS_NAME, "date").text)
        temperature_max.append(i.find_element(By.CLASS_NAME, "temp-hi").text)
        temperature_min.append(i.find_element(By.CLASS_NAME, "temp-lo").text)
        description_day.append(i.find_element(By.CLASS_NAME, "no-wrap").text)
        rain.append(i.find_element(By.CLASS_NAME, "precip").text)
    return date_day,temperature_max,temperature_min,description_day

In [5]:
def get_today_weather_details(driver):
    details_weather_today = driver.find_elements(By.CLASS_NAME, "daily-list-item ")[0]
    temp_max_today=details_weather_today.find_element(By.CLASS_NAME, "temp-hi").text
    temp_min_today=details_weather_today.find_element(By.CLASS_NAME, "temp-lo").text
    description_day=details_weather_today.find_element(By.CLASS_NAME, "no-wrap").text
    date_today=details_weather_today.find_element(By.CLASS_NAME, "date").text
    icon=details_weather_today.find_element(By.CLASS_NAME, "icon").get_attribute("src")
    href_value = details_weather_today.get_attribute("href")
    driver.get(href_value)
    div_element = driver.find_element(By.CLASS_NAME, "half-day-card-content")
    div_element1 = div_element.find_element(By.CLASS_NAME, "left")
    p_elements = div_element1.find_elements(By.TAG_NAME, "p")
    div_element_right= div_element.find_element(By.CLASS_NAME, "right")
    p_elements_right = div_element_right.find_elements(By.TAG_NAME, "p")
    return p_elements[1].text,p_elements[2].text,p_elements[3].text,p_elements_right[2].text,temp_max_today,temp_min_today,description_day,date_today,icon


In [6]:
def get_forcasting_hours(driver):
    link_images=[]
    hour=[]
    temps=[]
    precips=[]
    time.sleep(2)

    list_hours=driver.find_element(By.CLASS_NAME, "hourly-list__list")
    list_hours.find_element(By.XPATH,"/html/body/div/div[7]/div[1]/div[1]/div[2]/div/button[2]").click()
    time.sleep(4)
    hours=list_hours.find_elements(By.CLASS_NAME,"hourly-list__list__item-time")
    list_images=list_hours.find_elements(By.CLASS_NAME,"hourly-list__list__item-icon")
    temp=list_hours.find_elements(By.CLASS_NAME,"hourly-list__list__item-temp")
    precip=list_hours.find_elements(By.CLASS_NAME,"hourly-list__list__item-precip")
    for i in hours[1:6] :
        hour.append(i.text)
    for j in  list_images[1:6] :
        link_images.append(j.get_attribute("src"))
    for k in temp[1:6] :
        temps.append(k.text)
    for l in  precip[1:6] :
        precips.append(l.text)    
    return link_images,hour, temps, precips



In [7]:
cities=["sfax","monastir","Mahdia"]
def get_all_details(): 
    all_temp_max_today=[]
    all_temp_min_today=[]
    all_description_day_=[]
    all_date_today_=[]
    all_icon=[]
    sunrises=[]
    sunsets=[]
    vents=[]
    all_Précipitations_possibles=[]
    all_Rafales_vent=[]
    all_Couverture_nuageuse=[]
    all_date_today=[]
    all_temperature_max=[]
    all_temperature_min=[]
    all_descirpiton_day=[]
    all_link_images=[]
    all_hour=[]
    all_temps=[]
    all_precips=[]
    options = Options()
    options.headless = True
    driver = webdriver.Chrome(options=options)
    for i in cities:
        driver.get("https://www.accuweather.com/")
        input_element = driver.find_element(By.CLASS_NAME, "search-input")
        input_element.send_keys(i)
        input_element.send_keys(Keys.RETURN)
        time.sleep(5)  
        sunrise,sunset=get_sunrise_details(driver)
        sunrises.append(sunrise)
        sunsets.append(sunset)
        link_images,hour, temps, precips=get_forcasting_hours(driver)
        all_link_images.append(link_images)
        all_hour.append(hour)
        all_temps.append(temps)
        all_precips.append(precips)

        date_day,temperature_max,temperature_min,description_day=get_forcasting_details(driver)
        all_date_today.append(date_day)
        all_temperature_max.append(temperature_max)
        all_temperature_min.append(temperature_min)
        all_descirpiton_day.append(description_day)
        vent,Précipitations_possibles,Rafales_vent,Couverture_nuageuse,temp_max_today,temp_min_today,description_day_,date_today,icon=get_today_weather_details(driver)
        vents.append(vent)
        all_Précipitations_possibles.append(Précipitations_possibles)
        all_Rafales_vent.append(Rafales_vent)
        all_Couverture_nuageuse.append(Couverture_nuageuse)
        all_temp_max_today.append(temp_max_today)
        all_temp_min_today.append(temp_min_today)
        all_description_day_.append(description_day_)
        all_date_today_.append(date_today)
        all_icon.append(icon)
    return  sunrises,sunsets, vents,all_Précipitations_possibles,all_Rafales_vent,all_Couverture_nuageuse, all_date_today,all_temperature_max,all_temperature_min,all_descirpiton_day,all_link_images,all_hour,all_temps,all_precips,all_temp_max_today,all_temp_min_today,all_description_day_,all_date_today_,all_icon
  

In [8]:
def transform_df_forcast_hours(all_hour,all_temps,all_precips,all_link_images):    
    all_hour = [item for sublist in all_hour for item in sublist]
    all_temps = [item for sublist in all_temps for item in sublist]
    all_precips = [item for sublist in all_precips for item in sublist]
    all_link_images = [item for sublist in all_link_images for item in sublist]
    forcast_hours_df= pd.DataFrame({
    'heures':  all_hour, 
    'temperatures':all_temps,
    'precepitations':all_precips,
    'icones' :all_link_images,
    })
    num_rows = len(forcast_hours_df) // 3
    city_column = [city for city in cities for _ in range(num_rows)]
    forcast_hours_df.insert(0, 'Ville', city_column)
    return forcast_hours_df

In [9]:
def clean_dataframe_today_weather(df):
    columns_to_clean = ['Vent', 'Rafales de vent', 'Précipitations possibles', 'Couverture_nuageuse']
    
    for column in columns_to_clean:
        df[column] = df[column].apply(lambda x: x.split('\n')[-1] if isinstance(x, str) else x)
    
    return df


In [10]:
def convert_date_today(date_str, year):
    day_month = date_str.split('\n')[-1]
    day, month = day_month.split('/')
    real_date = datetime(year, int(month), int(day))
    return real_date.strftime('%m/%d/%Y')


In [11]:
def transform_today_weather_df(cities,sunrises,sunsets,vents,all_Précipitations_possibles,all_Couverture_nuageuse,all_Rafales_vent,all_temp_max_today,all_temp_min_today,all_description_day_,all_date_today_,all_icon):    
    today_weather_df= pd.DataFrame({
    'Ville':  cities, 
    'date':all_date_today_,
    'Temperature max':all_temp_max_today,
    'Temperature min':all_temp_min_today,
    'description':all_description_day_,
    'etat':all_icon,
    'Lever':sunrises,
    'Coucher':sunsets,
    'Vent' :vents,
    'Rafales de vent':all_Précipitations_possibles,
    'Précipitations possibles' :all_Rafales_vent,
    'Couverture_nuageuse':all_Couverture_nuageuse

    })
    today_weather_df = today_weather_df=clean_dataframe_today_weather(today_weather_df)
    today_weather_df['date'] = today_weather_df['date'].apply(lambda x: convert_date_today(x, 2024))

    return today_weather_df

In [12]:
def clean_temperature(temp_str):
        return re.sub(r'[^0-9]', '', temp_str)

In [13]:
def convert_dates(df):
    def parse_date(date_str):
        day_str, date_part = date_str.split('\n')
        day_mapping = {
            'LUN.': 'Monday',
            'MAR.': 'Tuesday',
            'MER.': 'Wednesday',
            'JEU.': 'Thursday',
            'VEN.': 'Friday',
            'SAM.': 'Saturday',
            'DIM.': 'Sunday'
        }
        
        # Convert date_part to the desired format
        date = datetime.strptime(date_part, '%d/%m').replace(year=2024)
        return date.strftime('%m/%d/%Y')

    df['dates'] = df['dates'].apply(parse_date)
    return df



In [14]:
def transform_forecasting_weather_df(all_temperature_max,all_date_today,all_temperature_min,all_descirpiton_day):   
    all_temperature_max_flat = [item for sublist in all_temperature_max for item in sublist]
    all_date_today_flat = [item for sublist in all_date_today for item in sublist]
    all_temperature_min_flat = [item for sublist in all_temperature_min for item in sublist]
    all_description_day_flat = [item for sublist in all_descirpiton_day for item in sublist]

    forecasting_weather_df = pd.DataFrame({
        "dates": all_date_today_flat,
        "temperature max": all_temperature_max_flat,
        "temperature min": all_temperature_min_flat,
        "descriptions": all_description_day_flat
    })
    forecasting_weather_df = convert_dates(forecasting_weather_df)
    forecasting_weather_df['temperature max'] = forecasting_weather_df['temperature max'].apply(clean_temperature)
    forecasting_weather_df['temperature min'] = forecasting_weather_df['temperature min'].apply(clean_temperature)
    num_rows = len(forecasting_weather_df) // 3
    city_column = [city for city in cities for _ in range(num_rows)]
    forecasting_weather_df.insert(0, 'Ville', city_column)
    return forecasting_weather_df

In [15]:
sunrises,sunsets, vents,all_Précipitations_possibles,all_Rafales_vent,all_Couverture_nuageuse, all_date_today,all_temperature_max,all_temperature_min,all_descirpiton_day,all_link_images,all_hour,all_temps,all_precips,all_temp_max_today,all_temp_min_today,all_description_day_,all_date_today_,all_icon=get_all_details()
forecasting_weather_df=transform_forecasting_weather_df(all_temperature_max,all_date_today,all_temperature_min,all_descirpiton_day)
today_weather_df=transform_today_weather_df(cities,sunrises,sunsets,vents,all_Précipitations_possibles,all_Couverture_nuageuse,all_Rafales_vent,all_temp_max_today,all_temp_min_today,all_description_day_,all_date_today_,all_icon)
forcast_hours_df=transform_df_forcast_hours(all_hour,all_temps,all_precips,all_link_images)

In [16]:
today_weather_df

,Ville,date,Temperature max,Temperature min,description,etat,Lever,Coucher,Vent,Rafales de vent,Précipitations possibles,Couverture_nuageuse
0,sfax,07/05/2024,30°,23°,Ensoleillé et agréable,https://www.awxcdn.com/adc-assets/images/weath...,05:10,19:34,ESE 13 km/h,37 km/h,0 %,3 %
1,monastir,07/05/2024,30°,22°,Ensoleillé et agréable; venteux l'après-midi,https://www.awxcdn.com/adc-assets/images/weath...,05:07,19:36,NE 22 km/h,39 km/h,0 %,0 %
2,Mahdia,07/05/2024,29°,22°,Ensoleillé et agréable,https://www.awxcdn.com/adc-assets/images/weath...,05:06,19:34,ENE 19 km/h,35 km/h,0 %,2 %


In [17]:
forcast_hours_df

,Ville,heures,temperatures,precepitations,icones
0,sfax,13 h,30°,0 %,https://www.awxcdn.com/adc-assets/images/weath...
1,sfax,14 h,29°,0 %,https://www.awxcdn.com/adc-assets/images/weath...
2,sfax,15 h,29°,0 %,https://www.awxcdn.com/adc-assets/images/weath...
3,sfax,16 h,28°,0 %,https://www.awxcdn.com/adc-assets/images/weath...
4,sfax,17 h,28°,0 %,https://www.awxcdn.com/adc-assets/images/weath...
5,monastir,13 h,28°,0 %,https://www.awxcdn.com/adc-assets/images/weath...
6,monastir,14 h,30°,0 %,https://www.awxcdn.com/adc-assets/images/weath...
7,monastir,15 h,29°,0 %,https://www.awxcdn.com/adc-assets/images/weath...
8,monastir,16 h,29°,0 %,https://www.awxcdn.com/adc-assets/images/weath...
9,monastir,17 h,28°,0 %,https://www.awxcdn.com/adc-assets/images/weath...


In [18]:
today_weather_df

,Ville,date,Temperature max,Temperature min,description,etat,Lever,Coucher,Vent,Rafales de vent,Précipitations possibles,Couverture_nuageuse
0,sfax,07/05/2024,30°,23°,Ensoleillé et agréable,https://www.awxcdn.com/adc-assets/images/weath...,05:10,19:34,ESE 13 km/h,37 km/h,0 %,3 %
1,monastir,07/05/2024,30°,22°,Ensoleillé et agréable; venteux l'après-midi,https://www.awxcdn.com/adc-assets/images/weath...,05:07,19:36,NE 22 km/h,39 km/h,0 %,0 %
2,Mahdia,07/05/2024,29°,22°,Ensoleillé et agréable,https://www.awxcdn.com/adc-assets/images/weath...,05:06,19:34,ENE 19 km/h,35 km/h,0 %,2 %


In [19]:
forecasting_weather_df

,Ville,dates,temperature max,temperature min,descriptions
0,sfax,07/06/2024,30,24,Ensoleillé et humide
1,sfax,07/07/2024,31,23,Ensoleillé et humide
2,sfax,07/08/2024,33,23,Soleil
3,sfax,07/09/2024,36,25,Ensoleillé et chaud
4,sfax,07/10/2024,37,25,Ensoleillé et chaud
5,sfax,07/11/2024,36,24,Ensoleillé et chaud
6,sfax,07/12/2024,34,23,Ensoleillé
7,sfax,07/13/2024,35,25,Ensoleillé et chaud
8,sfax,07/14/2024,35,24,Ensoleillé
9,monastir,07/06/2024,35,23,Soleil; venteux
